En esta primera practica lo que se busca es calculas las huellas hidrica, energetica y de GEI, de un proceso determinado. 

Hay algunas cosas que ya vienen explicitas en la presentación anterior por lo cual si tienen dudas soo consultenlas en alguna de las presentaciones que les deje. 

Para no tener inconvenientes con las versiones de bases de datos les dejo el link de la base de datos que usamos: https://nexus.openlca.org/database/ELCD 

In [ ]:
#Estos dos primeros modulos son con los que trabajamos dentro de OPEN LCA
import olca_ipc as ipc
import olca_schema as schema

import pandas as pd
from IPython.display import display

#Estos ultimos dos son con los cuales hacemos la conexion a OpenLCA
client = ipc.Client(8080)
client

In [ ]:
# Paso 1: Obtener el proceso
process_obj = client.get(schema.Process, name="Hydrogen chloride gas (HCl), production mix for PVC production, at plant")
# Paso 2: Obtener los inputs del proceso 
filtered_inputs = [exc for exc in process_obj.exchanges if exc.is_input]

In [ ]:
# Diccionario con los filtros que deseas aplicar
flow_filters = {
    "Water inputs": [exc for exc in filtered_inputs if 'water' in exc.flow.name.lower() and exc.flow.name != "Energy, primary, from water power"],
    "Energy inputs": [exc for exc in filtered_inputs if 'energy' in exc.flow.name.lower() and exc.flow.name != "energy (recovered)"],
    "CO2 inputs" : [exc for exc in process_obj.exchanges if not exc.is_input and ('carbon dioxide' in exc.flow.name.lower() or 'Methane' in exc.flow.name and exc.flow.name != "Methane, dichloro-, HCC-30")]

}

# Diccionario para guardar los DataFrames resultantes
df_results = {}

for label, flows in flow_filters.items():
    df = pd.DataFrame([{
        "Process": process_obj.name,
        "Name": exc.flow.name,
        "Amount": exc.amount
    } for exc in flows])

    # Agregar la fila con el total
    total_row = pd.DataFrame([{
        "Process": "",
        "Name": f"Total {label.lower()}",
        "Amount": df['Amount'].sum()
    }])

    df = pd.concat([df, total_row], ignore_index=True)
    
    # Guardar resultado
    df_results[label] = df

# Mostrar los resultados
for label, df in df_results.items():
    print(f"\n--- {label} ---")
    display(df)  # Usar print(df) si no estás en Jupyter

In [ ]:
flujo_a_modificar = "Methane"  # Nombre del flujo a modificar
factor_multiplicador = 28 # Factor por el cual quieres multiplicar la cantidad
# Acceder a la tabla que quieres modificar
df_CO2 = df_results["CO2 inputs"]


df_CO2 .loc[df_CO2 ["Name"] == flujo_a_modificar, "Amount"] *= factor_multiplicador
df_CO2 .loc[df_CO2 ["Name"].str.startswith("Total"), "Amount"] = df_CO2 [:-1]['Amount'].sum()# Actualizar 


df_results["CO2 inputs"] = df_CO2 
df_CO2 # Mostrar la tabla modificada

In [ ]:
with pd.ExcelWriter("_inputs_combined.xlsx", engine="xlsxwriter") as writer:
    row = 0  # Fila inicial

    for name, df in df_results.items():
        # Mostrar el nombre y la tabla en la notebook
        print(f"\nTabla: {name}")
        display(df)

        # Escribir el título en Excel
        df_title = pd.DataFrame({name: []})  # Título como encabezado
        df_title.to_excel(writer, sheet_name="Summary", startrow=row, index=False, header=False)
        row += 1

        # Escribir el DataFrame en Excel
        df.to_excel(writer, sheet_name="Summary", startrow=row, index=False)
        row += len(df) + 2  # Dejar espacio
